In [83]:
import requests
import pandas as pd
import herepy
import config

In [84]:
residents = pd.read_csv("addresses_simulated_incomes.csv")

car_mask = residents['simulated_commute_car']==1
residents_car = residents[car_mask]
residents_transit = residents[-car_mask]
#Need to add store and vacant lot locations into a single df. Idk where current store info is--can you add that and the
#vacant lot data here?
stores = residents

In [85]:
#limited rows for testing. Change to residents_car.loc[:, [..]].values instead of the slices to use whole datasets
start_car_coords = residents_car.loc[0:1,["LATITUDE", "LONGITUDE"]].values
start_transit_coords = residents_transit.loc[0:1,["LATITUDE", "LONGITUDE"]].values
end_coords = stores.loc[2:3,["LATITUDE", "LONGITUDE"]].values

routingApi = herepy.RoutingApi(config.api_key)

In [86]:
response_car = routingApi.matrix(
    start_waypoints=start_car_coords,
    destination_waypoints=end_coords,
    departure='2020-07-04T17:00:00+02',
    summary_attributes = ['traveltime'],
    modes=[herepy.RouteMode.fastest, herepy.RouteMode.car])

In [87]:
#The routingApi.matrix function doesn't support the publicTransport RouteMode-- will need to use the
#public_transport method below
response_transit = routingApi.matrix(
    start_waypoints=start_transit_coords,
    destination_waypoints=end_coords,
    departure='2020-07-04T17:00:00+02',
    summary_attributes = ['traveltime'],
    modes=[herepy.RouteMode.fastest, herepy.RouteMode.publicTransport])

response = routingApi.public_transport([11.0, 12.0],
                                       [15.0, 16.0],
                                       True,
                                       [herepy.RouteMode.publicTransport, herepy.RouteMode.fastest])

InvalidInputDataError: Transport type 'publicTransport' is not supported.

In [88]:
response_car.as_json_string()
#response_transit.as_json_string()

'{"response": {"matrixEntry": [{"destinationIndex": 0, "startIndex": 0, "summary": {"costFactor": 2428, "travelTime": 2348}}, {"destinationIndex": 1, "startIndex": 0, "summary": {"costFactor": 2100, "travelTime": 2040}}, {"destinationIndex": 0, "startIndex": 1, "summary": {"costFactor": 1322, "travelTime": 1282}}, {"destinationIndex": 1, "startIndex": 1, "summary": {"costFactor": 994, "travelTime": 974}}], "metaInfo": {"availableMapVersion": ["8.30.107.152"], "interfaceVersion": "2.6.76", "mapVersion": "8.30.107.152", "moduleVersion": "7.2.202015-6852", "timestamp": "2020-04-20T00:21:14Z"}}}'